In [37]:
%reset -f

In [39]:
import numpy as np
import sympy as sym
sym.init_printing(use_latex='mathjax')
import copy
import operator
eps = sym.symbols('epsilon')
lam = sym.symbols('lambda')
xlz = sym.Symbol('\\bar{x} + \\lambda \\bar{z} :')
from IPython.display import Latex, Math
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
from matplotlib.ticker import AutoMinorLocator, MultipleLocator
from scipy.spatial import ConvexHull, convex_hull_plot_2d
import itertools
import seaborn as sns; sns.set(); sns.set_style("whitegrid"); color_list = sns.color_palette("muted")

In [41]:
# perturb
def pivot_perturb():
    global m, b, Perturb, eps
    Perturb = True
    for i in range(m):
        for j in range(m):
            b[i] += A_beta[i,j]*eps**(j+1)
        print('pivot_perturb() done')

In [43]:
# algebra
def pivot_algebra():
    global m, n, objval, xbar, xbar_beta, xbar_eta, ybar, cbar_eta, ratios
    xbar_beta = A_beta.solve(b)
    xbar_eta = sym.zeros(n-m, 1)
    objval = c_beta.dot(xbar_beta)
    xbar = sym.zeros(n,1)
    for i in range(m): xbar[beta[i]] = xbar_beta[i]
    for j in range(n-m): xbar[eta[j]] = xbar_eta[j]
    ybar = A_beta.transpose().solve(c[beta,0])
    cbar_eta = c_eta - A_eta.transpose()*ybar
    ratios = sym.oo*sym.ones(m,1)
    print('pivot_algebra() done')

In [45]:
# numerical version of a d-by-1 array
def N(parray):
    for i in range(parray.shape[0]): display(sym.N(parray[i]))

In [47]:
# ratios (and direction) for a given nonbasic index eta_j
def pivot_ratios(j):
    global ratios, zbar
    if j>n-m-1:
        display(Latex("erro: $j$ is out of range."))
    else:
        A_etaj = copy.copy(A[:,eta[j]])
        Abar_etaj = A_beta.solve(A_etaj)
        for i in range(m):
            if Abar_etaj[i] > 0:
                ratios[i] = xbar_beta[i] / Abar_etaj[i]
            else:
                ratios[i] = sym.oo
        display(ratios)
        zbar = sym.zeros(n,1)
        for i in range(m): zbar[beta[i]] = -Abar_etaj[i]
        zbar[eta[j]] = 1
        display(xlz, xbar + lam*zbar)

In [49]:
# swap
def pivot_swap(j,i):
    global A_beta, A_eta, c_beta, c_eta
    if i > m-1 or j > n-m-1:
        display(Latex("error: $j$ or $i$ is out of range. swap not accepted"))
    else:
        save = copy.copy(beta[i])
        beta[i] = copy.copy(eta[j])
        eta[j] = save
        A_beta = copy.copy(A[:, beta])
        A_eta = copy.copy(A[:, eta])
        c_beta = copy.copy(c[beta,0])
        c_eta = copy.copy(c[eta,0])
        display(Latex("swap accepted --- new partition: "))
        print('eta: ', eta)
        print('beta: ', beta)
        print('*** MUST APPLY pivot_algebra()! ***')

In [65]:
A = sym.Matrix(([1,1],
                [2,1]))
m = A.shape[0]
n = A.shape[1]
c = sym.Matrix([[3],[2]])
b = sym.Matrix([[4], [5]])
beta = [0, 1]  
eta = [j for j in range(n) if j not in beta]

A_beta = A[:, beta]
A_eta = A[:, eta]
c_beta = c[beta, :]
c_eta = c[eta, :]
Perturb = True

In [67]:
A

⎡1  1⎤
⎢    ⎥
⎣2  1⎦

In [69]:
c

⎡3⎤
⎢ ⎥
⎣2⎦

In [73]:
pivot_perturb()
pivot_algebra()

pivot_perturb() done
pivot_perturb() done
pivot_algebra() done


In [77]:
# (i) Optimal Solution is reached
print("Example (i): Optimal Solution")
N(xbar)
display(Latex(f"Optimal Objective Value: {objval}"))

Example (i): Optimal Solution


2.0⋅ε + 1.0

     2      
2.0⋅ε  + 3.0

<IPython.core.display.Latex object>

In [83]:
## (ii) Unboundedness

ratios = sym.zeros(m, 1)
display(Latex("Ratios for checking unboundedness:"))

# Calculate the ratios for each non-basic variable
for j in range(len(eta)):
    A_etaj = A[:, eta[j]]
    Abar_etaj = A_beta.solve(A_etaj)
    
    for i in range(m):
        if Abar_etaj[i] > 0:
            ratios[i] = xbar_beta[i] / Abar_etaj[i]
        else:
            ratios[i] = sym.oo
            
    display(ratios)

# Check if any ratio is infinite (indicating unboundedness)
if any(ratio == sym.oo for ratio in ratios):
    display(Latex("The solution is unbounded."))
else:
    display(Latex("The solution is bounded."))

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>